In [45]:
import pandas as pd 
import numpy as np
from sklearn.model_selection import train_test_split

df = pd.read_csv("rai.csv")
df = df.sort_values(by=['popularity'])
df = df.drop_duplicates(subset=['id'])
df = df.drop_duplicates(subset=['name', 'main_artist'], keep='first')

In [46]:
df.describe()

,key,mode,time_signature,duration_ms_x,danceability,loudness,energy,instrumentalness,liveness,valence,speechiness,tempo,popularity,youtube_views
count,1691.000000,1691.000000,1691.000000,1691.000000,1691.000000,1691.000000,1691.000000,1691.000000,1691.000000,1691.000000,1691.000000,1691.000000,1691.000000,1.691000e+03
mean,5.311059,0.450621,3.896511,346177.345358,0.568270,-7.630745,0.753080,0.021678,0.206796,0.762262,0.085771,122.057871,5.551153,2.075357e+06
std,3.689292,0.497703,0.353252,100362.876571,0.135562,3.840073,0.160376,0.111717,0.170393,0.178160,0.065963,35.231541,7.362882,7.123416e+06
min,0.000000,0.000000,1.000000,49266.000000,0.196000,-27.008000,0.037000,0.000000,0.017300,0.036300,0.023500,53.985000,0.000000,0.000000e+00
25%,1.000000,0.000000,4.000000,290291.000000,0.473000,-9.957500,0.663500,0.000000,0.087550,0.668000,0.044000,93.963500,1.000000,3.735000e+03
50%,6.000000,0.000000,4.000000,342439.000000,0.590000,-6.973000,0.789000,0.000012,0.143000,0.810000,0.062000,103.194000,3.000000,7.147800e+04
75%,9.000000,1.000000,4.000000,388093.000000,0.674000,-4.949500,0.873500,0.000320,0.290000,0.898500,0.098550,158.877000,8.000000,9.476350e+05
max,11.000000,1.000000,5.000000,945016.000000,0.883000,1.108000,0.994000,0.939000,0.982000,0.983000,0.525000,206.878000,47.000000,1.175880e+08


## Feature engineering

In [47]:
df['popularity_cat'] = 0
df.loc[(df['popularity'] < 8), 'popularity_cat' ] = 0 
df.loc[(df['popularity'] >= 8), 'popularity_cat'] = 1

## Split & prepare the data

In [48]:
train, test = train_test_split(df, test_size=0.2)

selected_features = ["key", "mode", "time_signature", "duration_ms_x","danceability", "loudness", "energy", "instrumentalness", "liveness", "valence", "speechiness", "tempo"] 
selected_target = "popularity_cat"

train_x = train[selected_features] 
train_y = train[selected_target]

test_x = test[selected_features] 
test_y = test[selected_target]

## Building the model

In [49]:
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier(n_estimators = 10)

In [50]:
forest.fit(X = train_x, y = train_y)

RandomForestClassifier(n_estimators=10)

In [51]:
from sklearn.metrics import make_scorer, accuracy_score, roc_auc_score 
rf_predict = forest.predict(test_x)
accuracy_score(test_y, rf_predict)

0.7286135693215339

In [52]:
test["prediction"] = rf_predict
test.head()

,id,name,main_artist,key,mode,time_signature,duration_ms_x,danceability,loudness,energy,instrumentalness,liveness,valence,speechiness,tempo,popularity,youtube_views,popularity_cat,prediction
673,39S81pUG3BUeCpACD7iAXE,Nagouad Nabeghik,Cheb Djalil,9,1,4,418841,0.448,-8.806,0.848,0.002850,0.1420,0.913,0.0685,89.930,7,1595,0,0
805,0KdtSvEZFB1wvBNzo9d2bW,لوكان يغيب غير نهار,Warda,8,0,4,455706,0.589,-8.281,0.831,0.000388,0.1000,0.951,0.0848,93.899,6,286934,0,0
1452,3thILVL2mKnzDCZafm0OTR,ROUKED EL ACHRA,Cheikha Rimitti,7,0,4,706280,0.533,-5.147,0.920,0.000000,0.3800,0.541,0.1020,171.705,3,9739,0,0
7,0GDjF1aq0W3hZN9whZtcmw,Al Hachwa Hachwa,Cheb Bello,4,1,4,315715,0.638,-5.637,0.883,0.000010,0.0814,0.965,0.0423,89.898,42,50406176,1,1
1957,0TuMU1h5dlqSx3kzL8csf2,Bafana bafana,Cheb Bilal,1,1,4,286217,0.688,-3.727,0.932,0.000003,0.8090,0.722,0.0440,135.969,1,130121,0,0


## Plotting an estimator

In [53]:
estimator = forest.estimators_[5]

from sklearn.tree import export_graphviz
# Export as dot file
export_graphviz(estimator, out_file='tree.dot', 
                feature_names = selected_features,
                class_names = selected_target,
                rounded = True, proportion = False, 
                precision = 2, filled = True)

import os
os.system('dot -Tpng tree.dot -o tree.png')


1